## Dataset Link: https://www.kaggle.com/mlg-ulb/creditcardfraud

Context

It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.

Content

The dataset contains transactions made by credit cards in September 2013 by European cardholders. 
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

In [4]:
## required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

pd.set_option('display.max_columns', None)
sns.set_style('darkgrid')

In [36]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with Novchan_ML


In [39]:
default_ds = ws.get_default_datastore()

if 'credit dataset' not in ws.datasets:
    ## first upload the files
    default_ds.upload_files(files=['./creditcard.csv'], # Upload the csv files in /data
                        target_path='Fraud_Detection/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)
    
        #Then Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'Fraud_Detection/*.csv'))

    # Then Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='credit dataset',
                                description='For Credit Card Fraud Detection',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


In [41]:
# Create a folder for the experiment files
experiment_folder = 'Credit_card_fraud_detection_on_azure'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

Credit_card_fraud_detection_on_azure folder created


In [42]:
%%writefile $experiment_folder/fraud_detection_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,precision_score

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=1, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter
reg = args.reg_rate ## set reg based on what we input in ScriptRunConfig argument parameter

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
fraud_detection = run.input_datasets['training_data'].to_pandas_dataframe() ##<-recall, select datasets using name instead of id (see 06)

#split data into training set and test set
X = fraud_detection.iloc[:, 1:-2]
y = fraud_detection.iloc[:, -1]

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate precision
y_hat = model.predict(X_test)
precision = precision_score(y_test,y_hat)
print('Precision:', precision)
run.log('Precision', np.float(precision))

## Plot the imbalanced data
fig = plt.figure(figsize=(12,7)) ## seems it cant change the size of the FacetGrid
#sns.catplot(data=fraud_detection,x='Class',kind='count',log=True) ##<-for unknown reason, azure cannot log FacetGrid (shows blank)
sns.countplot(data=fraud_detection,x='Class',log=True)
# g.fig.suptitle("Class Imbalance",y=1.05,fontsize=15,horizontalalignment='right') #<-also works!
plt.title("Test",y=1.05,fontsize=15,loc='left')
plt.xticks([0,1],['False','True'],size=20)
plt.xlabel(xlabel='CLASS',size=20) #<-xlabel arg is required
run.log_image(name="Imbalanced_data",plot=fig)
plt.show()

os.makedirs('outputs',exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/fraud_detection_model.pkl') ##<-recall, as this py is run on cloud, you can see this file in the outputs tab in Studio

run.complete()

Overwriting Credit_card_fraud_detection_on_azure/fraud_detection_training.py


In [47]:
%%writefile $experiment_folder/fraud_detection_training_w_SMOTE.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,precision_score, recall_score
from imblearn.over_sampling import SMOTE


# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=1, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter
reg = args.reg_rate ## set reg based on what we input in ScriptRunConfig argument parameter

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
fraud_detection = run.input_datasets['training_data'].to_pandas_dataframe() ##<-recall, select datasets using name instead of id (see 06)

#split data into training set and test set
X = fraud_detection.iloc[:, 1:-2]
y = fraud_detection.iloc[:, -1]

#instantiate SMOTE and transform into balanced dataset
sm = SMOTE(random_state=42)
X_bal,y_bal = sm.fit_resample(X,y)

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, 
                                                    test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate precision
y_hat = model.predict(X_test)
precision = precision_score(y_test,y_hat)
print('Precision:', precision)
run.log('Precision', np.float(precision))

#calculate recall
recall = recall_score(y_test,y_hat)
print('recall:',recall)
run.log('Recall',np.float(recall))

## Plot the imbalanced data
fig = plt.figure(figsize=(12,7)) ## seems it cant change the size of the FacetGrid
#sns.catplot(data=fraud_detection,x='Class',kind='count',log=True) ##<-for unknown reason, azure cannot log FacetGrid (shows blank)
sns.countplot(data=fraud_detection,x='Class',log=True)
# g.fig.suptitle("Class Imbalance",y=1.05,fontsize=15,horizontalalignment='right') #<-also works!
plt.title("Class Distribution",y=1.05,fontsize=15,loc='center')
plt.xticks([0,1],['False','True'],size=10)
plt.xlabel(xlabel='CLASS',size=10) #<-xlabel arg is required
run.log_image(name="Imbalanced_data",plot=fig)
plt.show()

## Plot the balanced data
fig = plt.figure(figsize=(12,7))
y_bal.value_counts().plot(kind='bar')
plt.xticks([0,1],['False','True'],rotation=0,size=15)
plt.yticks(size=15)
plt.xlabel("Class",size=15)
plt.title("Class distribution",loc='center',size=20)
run.log_image(name="Balanced_data",plot=fig)
plt.show()


os.makedirs('outputs',exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/fraud_detection_model_2.pkl') ##<-recall, as this py is run on cloud, you can see this file in the outputs tab in Studio

run.complete()

Overwriting Credit_card_fraud_detection_on_azure/fraud_detection_training_w_SMOTE.py


In [68]:
%%writefile $experiment_folder/fraud_detection_training_w_SMOTE_HYPER.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,precision_score, recall_score, roc_auc_score
from imblearn.over_sampling import SMOTE


# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=1, help='regularization rate')
parser.add_argument('--penalty',type=str,dest='penalty',default='none',help='penalty')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameters
reg = args.reg_rate ## set reg based on what we input in ScriptRunConfig argument parameter
pen = args.penalty

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
fraud_detection = run.input_datasets['training_data'].to_pandas_dataframe() ##<-recall, select datasets using name instead of id (see 06)

#split data into training set and test set
X = fraud_detection.iloc[:, 1:-2]
y = fraud_detection.iloc[:, -1]

#instantiate SMOTE and transform into balanced dataset
sm = SMOTE(random_state=42)
X_bal,y_bal = sm.fit_resample(X,y)

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, 
                                                    test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="lbfgs",penalty=pen).fit(X_train, y_train)

# calculate precision
y_hat = model.predict(X_test)
precision = precision_score(y_test,y_hat)
print('Precision:', precision)
run.log('Precision', np.float(precision))

#calculate recall
recall = recall_score(y_test,y_hat)
print('recall:',recall)
run.log('Recall',np.float(recall))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

## Plot the imbalanced data
fig = plt.figure(figsize=(12,7)) ## seems it cant change the size of the FacetGrid
#sns.catplot(data=fraud_detection,x='Class',kind='count',log=True) ##<-for unknown reason, azure cannot log FacetGrid (shows blank)
sns.countplot(data=fraud_detection,x='Class',log=True)
# g.fig.suptitle("Class Imbalance",y=1.05,fontsize=15,horizontalalignment='right') #<-also works!
plt.title("Class Distribution",y=1.05,fontsize=15,loc='center')
plt.xticks([0,1],['False','True'],size=10)
plt.xlabel(xlabel='CLASS',size=10) #<-xlabel arg is required
run.log_image(name="Imbalanced_data",plot=fig)
plt.show()

## Plot the balanced data
fig = plt.figure(figsize=(12,7))
y_bal.value_counts().plot(kind='bar')
plt.xticks([0,1],['False','True'],rotation=0,size=15)
plt.yticks(size=15)
plt.xlabel("Class",size=15)
plt.title("Class distribution",loc='center',size=20)
run.log_image(name="Balanced_data",plot=fig)
plt.show()


os.makedirs('outputs',exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/fraud_detection_model_2.pkl') ##<-recall, as this py is run on cloud, you can see this file in the outputs tab in Studio

run.complete()

Overwriting Credit_card_fraud_detection_on_azure/fraud_detection_training_w_SMOTE_HYPER.py


In [53]:
%%writefile $experiment_folder/experiment_env_2.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- seaborn
- pip
- pip:
  - azureml-defaults  ##<-pip dependencies
  - pyarrow           ##<-pip dependencies
  - imbalanced-learn

Overwriting Credit_card_fraud_detection_on_azure/experiment_env_2.yml


In [54]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env_2.yml")
                                                  ## name of the env (should match that in the yml file?)

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())
                                            ## Serialize conda dependencies object into a string. 
                                            ## effectively printing the yml content

experiment_env defined.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- seaborn
- pip
- pip:
  - azureml-defaults  ##<-pip dependencies
  - pyarrow           ##<-pip dependencies
  - imbalanced-learn



In [45]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "NC-cluster-230102"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    ## ComputeTarget class is for compute cluster creation -->https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python
    ## ComputeInstance class if to create compute instance -->https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-manage-compute-instance?tabs=python
    ## but both classes hv ComputeTargetException
    ## difference:  ComputeInstance.provisioning_configuration()  vs AmlCompute.provisioning_configuration() for cluster
    ## difference2: ComputeInstance.create() vs ComputeTarget.create() for cluster

    print('Found existing cluster, use it.')

except ComputeTargetException: ##<-this exception will raise if there is no such compute target
    # If it doesn't already exist, create it
    try:
        ## setting the configuration
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)


        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [55]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
# from azureml-widgets import RunDetails

# Get the training dataset
credit_ds = ws.datasets.get("credit dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='fraud_detection_training_w_SMOTE.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', credit_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env,
                                compute_target=cluster_name)
                                
# submit the experiment
experiment_name = 'mslearn-train-fraud-detection-23-01-02'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
# RunDetails(run).show()
run.wait_for_completion()

{'runId': 'mslearn-train-fraud-detection-23-01-02_1672640094_0e655e11',
 'target': 'NC-cluster-230102',
 'status': 'Completed',
 'startTimeUtc': '2023-01-02T06:27:15.449063Z',
 'endTimeUtc': '2023-01-02T06:28:43.045246Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '6f636c8a-371c-48f3-aaeb-b9e15d4662f2',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'aeb96ed1-60be-48f5-aa46-abca656031b8'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'training_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'fraud_detection_training_w_SMOTE.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--regularization',
   '0.1',
   '--input-data',
   'DatasetConsumptionConfig:training_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 

In [56]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names(): ##<-recall, it shows all the thnigs in the outputs+logs tab
    print(file)

Regularization Rate 0.1
Precision 0.9730893701767488
Recall 0.9152510624348783
Imbalanced_data aml://artifactId/ExperimentRun/dcid.mslearn-train-fraud-detection-23-01-02_1672640094_0e655e11/Imbalanced_data_1672640906.png
Balanced_data aml://artifactId/ExperimentRun/dcid.mslearn-train-fraud-detection-23-01-02_1672640094_0e655e11/Balanced_data_1672640907.png


Balanced_data_1672640907.png
Imbalanced_data_1672640906.png
azureml-logs/20_image_build_log.txt
logs/azureml/dataprep/0/backgroundProcess.log
logs/azureml/dataprep/0/backgroundProcess_Telemetry.log
logs/azureml/dataprep/0/rslex.log.2023-01-02-06
outputs/fraud_detection_model_2.pkl
system_logs/cs_capability/cs-capability.log
system_logs/hosttools_capability/hosttools-capability.log
system_logs/lifecycler/execution-wrapper.log
system_logs/lifecycler/lifecycler.log
system_logs/metrics_capability/metrics-capability.log
system_logs/snapshot_capability/snapshot-capability.log
user_logs/std_log.txt


In [69]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.core.runconfig import DockerConfiguration
# from azureml-widgets import RunDetails

# Get the training dataset
credit_ds = ws.datasets.get("credit dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='fraud_detection_training_w_SMOTE_HYPER.py',
                                arguments = ['--input-data', credit_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env,
                                compute_target=cluster_name)
                                
# Sample a range of parameter values
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--regularization': choice(0.1, 1.0,10),
        '--penalty' : choice(['none', 'l2'])
    }
)                                

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

                                
# submit the experiment
experiment_name = 'mslearn-train-fraud-detection-23-01-02_hyper_tune'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=hyperdrive)
# RunDetails(run).show()
run.wait_for_completion()



{'runId': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a',
 'target': 'NC-cluster-230102',
 'status': 'Completed',
 'startTimeUtc': '2023-01-02T07:31:21.817958Z',
 'endTimeUtc': '2023-01-02T07:35:36.601639Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '392a7afe-6652-42db-8210-834e957b154a',
  'user_agent': 'python/3.9.7 (macOS-10.16-x86_64-i386-64bit) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': '6',
  'score': '0.9882588838202568',
  'best_child_run_id': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-s

In [71]:
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Arguments:',script_arguments)

{'run_id': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_0', 'hyperparameters': '{"--regularization": 0.1, "--penalty": "l2"}', 'best_primary_metric': 0.9882588838202568, 'status': 'Completed'}
{'run_id': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_5', 'hyperparameters': '{"--regularization": 10, "--penalty": "none"}', 'best_primary_metric': 0.988258806158486, 'status': 'Completed'}
{'run_id': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_3', 'hyperparameters': '{"--regularization": 1.0, "--penalty": "none"}', 'best_primary_metric': 0.988258806158486, 'status': 'Completed'}
{'run_id': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_1', 'hyperparameters': '{"--regularization": 0.1, "--penalty": "none"}', 'best_primary_metric': 0.988258806158486, 'status': 'Completed'}
{'run_id': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_2', 'hyperparameters': '{"--regularization": 1.0, "--penalty": "l2"}', 'best_primary_metric': 0.9882582047952175, 'status': 'Completed'}
{'run_id': 'HD_9cbfbf3b-1b91-48b7-93ef-0db373b4568a_4